# Query the Dataset in S3 using Amazon Athena and AWS Glue Catalog
This will create an Athena table in the Glue Catalog (Hive Metastore) which allows us to query the dataset files in S3.  We will create a table in the Glue Catalog based on the `Amazon Customer Reviews Dataset` in S3.  We define the columns that map to the data, specify how the data is formatted, and provide the location in Amazon S3 for the file(s).


<img src="img/athena_register_parquet.png" width="60%" align="left">

In [1]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

In [2]:
from pyathena import connect
import pandas as pd

In [3]:
%store -r setup_dependencies_passed

In [4]:
try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [5]:
print(setup_dependencies_passed)

True


In [6]:
if not setup_dependencies_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


# Create Athena Table for Parquet Data

#### Dataset columns

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.



In [7]:
s3_staging_dir = "s3://{}/athena/query-cache".format(bucket)

In [8]:
database_name = "default"
table_name = "amazon_reviews_parquet"

In [9]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [10]:
# SQL statement to execute
statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date bigint, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://amazon-reviews-pds/parquet/'
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)


    CREATE EXTERNAL TABLE IF NOT EXISTS default.amazon_reviews_parquet (
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date bigint, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://amazon-reviews-pds/parquet/'



""


# Analyze and Build the Partitions

In [11]:
statement = """
    MSCK REPAIR TABLE {}.{}
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)


    MSCK REPAIR TABLE default.amazon_reviews_parquet



""


# Verify The Table Has Been Created Succesfully

In [12]:
statement = "SHOW TABLES in {}".format(database_name)

df_tables = pd.read_sql(statement, conn)
df_tables

,tab_name
0,amazon_reviews_parquet


# Run A Sample Query

In [13]:
product_category = "Digital_Software"

statement = """
    SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100
""".format(
    database_name, table_name, product_category
)

print(statement)


    SELECT * FROM default.amazon_reviews_parquet
    WHERE product_category = 'Digital_Software' LIMIT 100



In [14]:
df = pd.read_sql(statement, conn)
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,year,product_category
0,US,24954530,R2DQXB8NBLNE3W,B00C79J2YM,789114735,Wondershare Video Converter Pro [Download],1,3,4,N,Y,DO NOT BUY!! PIRATED SOFTWARE!!,Bought this for instant download and when I tr...,15899,2013,Digital_Software
1,US,45559297,RHJ1R1KLPUPIZ,B00E7X9RUK,109813655,Quicken Deluxe 20,4,0,0,N,Y,This is a good program and I have used quicken...,This is a good program and I have used quicken...,16391,2014,Digital_Software
2,US,40495668,RIVSJQTLD3FJC,B00CQ7OF6A,102841476,RPG Maker VX Ace,4,0,0,N,N,Excellent Software,"The RPG Maker software is great, especially fo...",15899,2013,Digital_Software
3,US,24967696,R2VZUNPX142ZMW,B002PDPIF2,869467193,Quicken Deluxe 2010 [OLD VERSION],5,0,0,N,Y,Five Stars,I like it very much. I'ts everything it shoul...,16391,2014,Digital_Software
4,US,38736324,R1HJR4681HLF1Q,B005NBNU26,700466444,Smith Micro Software Poser Debut,4,8,8,N,Y,This is basically Poser 8. Utility depends on ...,The download version worked just fine. It took...,15900,2013,Digital_Software


In [15]:
if df.empty:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA.")
    print("LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.             ")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [16]:
if "amazon_reviews_parquet" in df_tables.values and not df.empty:
    ingest_create_athena_table_parquet_passed = True

In [17]:
%store ingest_create_athena_table_parquet_passed

Stored 'ingest_create_athena_table_parquet_passed' (bool)


# Review the New Athena Table in the Glue Catalog

In [18]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#table:catalog={};name=amazon_reviews_parquet;namespace=default">AWS Glue Catalog</a></b>'.format(
            region, account_id
        )
    )
)

# Store Variables for the Next Notebooks

In [19]:
%store

Stored variables and their in-db values:
ingest_create_athena_table_parquet_passed             -> True
setup_dependencies_passed                             -> True


# Release Resources

In [20]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>